### Assignment 1. Do regression task on MoleculeNet/ESOL with GCN, GraphSAGE, GIN, GAT, GT

In [6]:
from torch_geometric.datasets import MoleculeNet
from rdkit import Chem
from rdkit.Chem import Draw, AllChem
import matplotlib.pyplot as plt
import py3Dmol


# Load MoleculeNet ESOL dataset
dataset = MoleculeNet(root="data/MoleculeNet", name="ESOL")

# Extract the SMILES strings from the dataset (Tox21 contains SMILES in `smiles` attribute)
smiles_list = dataset.data.smiles  # SMILES strings are typically in this field

# Process the first molecule
example_index = 1
smiles = smiles_list[example_index]
print(f"SMILES String: {smiles}")

# Convert SMILES to RDKit molecule
mol = Chem.MolFromSmiles(smiles)


SMILES String: Cc1occc1C(=O)Nc2ccccc2


/home/user/anaconda3/envs/GNN2/lib/python3.9/site-packages/torch_geometric/data/in_memory_dataset.py:300: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  warnings.warn(msg)


In [7]:
from torch_geometric.datasets import MoleculeNet
from torch_geometric.loader import DataLoader
import torch

dataset = MoleculeNet(root='/tmp/MoleculeNet', name='ESOL')

train_dataset = dataset[:int(len(dataset) * 0.8)]
val_dataset = dataset[int(len(dataset) * 0.8):int(len(dataset) * 0.9)]
test_dataset = dataset[int(len(dataset) * 0.9):]

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)
test_loader = DataLoader(test_dataset, batch_size=32)


Processing...
Done!


In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, SAGEConv, GINConv, GATConv, global_mean_pool, TransformerConv
from torch_geometric.data import Data

# GCN
class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers=2, dropout=0.5):
        super(GCN, self).__init__()
        self.convs = torch.nn.ModuleList()
        self.batch_norms = torch.nn.ModuleList()

        # Input layer
        self.convs.append(GCNConv(in_channels, hidden_channels))
        self.batch_norms.append(torch.nn.BatchNorm1d(hidden_channels))

        # Hidden layers
        for _ in range(num_layers - 1):
            self.convs.append(GCNConv(hidden_channels, hidden_channels))
            self.batch_norms.append(torch.nn.BatchNorm1d(hidden_channels))

        # Output layer
        self.fc = nn.Linear(hidden_channels, out_channels)
        self.dropout = torch.nn.Dropout(p=dropout)

    def forward(self, x, edge_index, batch):
        for conv, batch_norm in zip(self.convs, self.batch_norms):
            x = conv(x, edge_index)
            x = batch_norm(x)
            x = F.relu(x)
            x = self.dropout(x)

        x = global_mean_pool(x, batch)  # Global mean pooling
        x = self.fc(x)
        return x

# GraphSAGE
class GraphSAGE(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers=2, dropout=0.5):
        super(GraphSAGE, self).__init__()
        self.convs = torch.nn.ModuleList()
        self.batch_norms = torch.nn.ModuleList()

        self.convs.append(SAGEConv(in_channels, hidden_channels))
        self.batch_norms.append(torch.nn.BatchNorm1d(hidden_channels))

        for _ in range(num_layers - 1):
            self.convs.append(SAGEConv(hidden_channels, hidden_channels))
            self.batch_norms.append(torch.nn.BatchNorm1d(hidden_channels))

        self.fc = nn.Linear(hidden_channels, out_channels)
        self.dropout = torch.nn.Dropout(p=dropout)

    def forward(self, x, edge_index, batch):
        for conv, batch_norm in zip(self.convs, self.batch_norms):
            x = conv(x, edge_index)
            x = batch_norm(x)
            x = F.relu(x)
            x = self.dropout(x)

        x = global_mean_pool(x, batch)
        x = self.fc(x)
        return x

# GIN 
class GIN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers=2, dropout=0.5):
        super(GIN, self).__init__()
        self.convs = torch.nn.ModuleList()
        self.batch_norms = torch.nn.ModuleList()

        self.convs.append(GINConv(nn=nn.Sequential(
            nn.Linear(in_channels, hidden_channels),
            nn.ReLU(),
            nn.Linear(hidden_channels, hidden_channels)
        )))
        self.batch_norms.append(torch.nn.BatchNorm1d(hidden_channels))

        for _ in range(num_layers - 1):
            self.convs.append(GINConv(nn=nn.Sequential(
                nn.Linear(hidden_channels, hidden_channels),
                nn.ReLU(),
                nn.Linear(hidden_channels, hidden_channels)
            )))
            self.batch_norms.append(torch.nn.BatchNorm1d(hidden_channels))

        self.fc = nn.Linear(hidden_channels, out_channels)
        self.dropout = torch.nn.Dropout(p=dropout)

    def forward(self, x, edge_index, batch):
        for conv, batch_norm in zip(self.convs, self.batch_norms):
            x = conv(x, edge_index)
            x = batch_norm(x)
            x = F.relu(x)
            x = self.dropout(x)

        x = global_mean_pool(x, batch)
        x = self.fc(x)
        return x

# GAT 
class GAT(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers=2, dropout=0.5):
        super(GAT, self).__init__()
        self.convs = torch.nn.ModuleList()
        self.batch_norms = torch.nn.ModuleList()

        self.convs.append(GATConv(in_channels, hidden_channels))
        self.batch_norms.append(torch.nn.BatchNorm1d(hidden_channels))

        for _ in range(num_layers - 1):
            self.convs.append(GATConv(hidden_channels, hidden_channels))
            self.batch_norms.append(torch.nn.BatchNorm1d(hidden_channels))

        self.fc = nn.Linear(hidden_channels, out_channels)
        self.dropout = torch.nn.Dropout(p=dropout)

    def forward(self, x, edge_index, batch):
        for conv, batch_norm in zip(self.convs, self.batch_norms):
            x = conv(x, edge_index)
            x = batch_norm(x)
            x = F.relu(x)
            x = self.dropout(x)

        x = global_mean_pool(x, batch)
        x = self.fc(x)
        return x

# GT (Graph Transformer) 
class GT(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers=2, dropout=0.5):
        super(GT, self).__init__()
        self.convs = torch.nn.ModuleList()
        self.batch_norms = torch.nn.ModuleList()

        # Use TransformerConv for GT
        self.convs.append(TransformerConv(in_channels, hidden_channels))
        self.batch_norms.append(torch.nn.BatchNorm1d(hidden_channels))

        for _ in range(num_layers - 1):
            self.convs.append(TransformerConv(hidden_channels, hidden_channels))
            self.batch_norms.append(torch.nn.BatchNorm1d(hidden_channels))

        self.fc = nn.Linear(hidden_channels, out_channels)
        self.dropout = torch.nn.Dropout(p=dropout)

    def forward(self, x, edge_index, batch):
        for conv, batch_norm in zip(self.convs, self.batch_norms):
            x = conv(x, edge_index)
            x = batch_norm(x)
            x = F.relu(x)
            x = self.dropout(x)

        x = global_mean_pool(x, batch)
        x = self.fc(x)
        return x


In [4]:
# Define the GCN model
class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers=2, dropout=0.5):
        super(GCN, self).__init__()
        self.convs = torch.nn.ModuleList()
        self.batch_norms = torch.nn.ModuleList()
        
        # Input layer
        self.convs.append(GCNConv(in_channels, hidden_channels))
        self.batch_norms.append(torch.nn.BatchNorm1d(hidden_channels))
        
        # Hidden layers
        for _ in range(num_layers - 1):
            self.convs.append(GCNConv(hidden_channels, hidden_channels))
            self.batch_norms.append(torch.nn.BatchNorm1d(hidden_channels))
        
        # Output layer
        self.fc = Linear(hidden_channels, out_channels)
        self.dropout = torch.nn.Dropout(p=dropout)
    
    def forward(self, x, edge_index, batch):
        # Apply GCN layers
        for conv, batch_norm in zip(self.convs, self.batch_norms):
            x = conv(x, edge_index)
            x = batch_norm(x)
            x = F.relu(x)
            x = self.dropout(x)
        
        # Global pooling
        x = global_mean_pool(x, batch)
        x = self.fc(x)
        return x

In [9]:
import torch.optim as optim
from sklearn.metrics import mean_squared_error, mean_absolute_error
import torch.nn.functional as F

criterion = torch.nn.MSELoss()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = GCN(
    in_channels=dataset.num_node_features,
    hidden_channels=64,
    out_channels=1,  
    num_layers=3,
    dropout=0.5
).to(device)

optimizer = optim.Adam(model.parameters(), lr=0.001)

# 학습 함수
def train():
    model.train()
    total_loss = 0
    for data in train_loader:
        data = data.to(device)
        data.x = data.x.float()
        data.y = data.y.float()

        optimizer.zero_grad()
        out = model(data.x, data.edge_index, data.batch)
        loss = criterion(out, data.y.view(-1, 1))  
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(train_loader)

import torch
from sklearn.metrics import mean_squared_error, mean_absolute_error

def evaluate(loader):
    model.eval()
    total_loss = 0
    y_true, y_pred = [], []
    with torch.no_grad():
        for data in loader:
            data = data.to(device)
            data.x = data.x.float()
            data.y = data.y.float()

            out = model(data.x, data.edge_index, data.batch)
            loss = criterion(out, data.y.view(-1, 1))
            total_loss += loss.item()

            y_true.append(data.y.cpu())
            y_pred.append(out.cpu())

    y_true = torch.cat(y_true, dim=0)
    y_pred = torch.cat(y_pred, dim=0)

    mse = mean_squared_error(y_true.numpy(), y_pred.numpy())
    rmse = torch.sqrt(torch.tensor(mse))  
    mae = mean_absolute_error(y_true.numpy(), y_pred.numpy())

    return total_loss / len(loader), rmse.item(), mae  

In [10]:
for epoch in range(1, 31):
    train_loss = train()
    val_loss, val_rmse, val_mae = evaluate(val_loader)
    print(f"Epoch: {epoch:02d}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val RMSE: {val_rmse:.4f}, Val MAE: {val_mae:.4f}")

test_loss, test_rmse, test_mae = evaluate(test_loader)
print(f"Test Loss: {test_loss:.4f}, Test RMSE: {test_rmse:.4f}, Test MAE: {test_mae:.4f}")


Epoch: 01, Train Loss: 12.9826, Val Loss: 11.7250, Val RMSE: 3.4540, Val MAE: 2.8551
Epoch: 02, Train Loss: 9.6557, Val Loss: 8.1820, Val RMSE: 2.8917, Val MAE: 2.3359
Epoch: 03, Train Loss: 6.8217, Val Loss: 6.2527, Val RMSE: 2.5288, Val MAE: 2.0552
Epoch: 04, Train Loss: 5.1686, Val Loss: 4.6560, Val RMSE: 2.1800, Val MAE: 1.7869
Epoch: 05, Train Loss: 4.1268, Val Loss: 3.6647, Val RMSE: 1.9289, Val MAE: 1.5969
Epoch: 06, Train Loss: 3.8822, Val Loss: 3.3258, Val RMSE: 1.8311, Val MAE: 1.5233
Epoch: 07, Train Loss: 3.2430, Val Loss: 2.8203, Val RMSE: 1.6822, Val MAE: 1.4011
Epoch: 08, Train Loss: 3.1145, Val Loss: 6.3690, Val RMSE: 2.5333, Val MAE: 2.0701
Epoch: 09, Train Loss: 2.7074, Val Loss: 2.8670, Val RMSE: 1.6976, Val MAE: 1.4134
Epoch: 10, Train Loss: 2.4434, Val Loss: 1.9120, Val RMSE: 1.3758, Val MAE: 1.0661
Epoch: 11, Train Loss: 2.4106, Val Loss: 1.5712, Val RMSE: 1.2720, Val MAE: 1.0082
Epoch: 12, Train Loss: 2.2700, Val Loss: 1.5378, Val RMSE: 1.2470, Val MAE: 0.9937
Ep

In [11]:
import torch.optim as optim
from sklearn.metrics import mean_squared_error, mean_absolute_error
import torch.nn.functional as F

criterion = torch.nn.MSELoss()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = GraphSAGE(
    in_channels=dataset.num_node_features,
    hidden_channels=64,
    out_channels=1,  
    num_layers=3,
    dropout=0.5
).to(device)

optimizer = optim.Adam(model.parameters(), lr=0.001)

# 학습 함수
def train():
    model.train()
    total_loss = 0
    for data in train_loader:
        data = data.to(device)
        data.x = data.x.float()
        data.y = data.y.float()

        optimizer.zero_grad()
        out = model(data.x, data.edge_index, data.batch)
        loss = criterion(out, data.y.view(-1, 1))  
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(train_loader)

import torch
from sklearn.metrics import mean_squared_error, mean_absolute_error

def evaluate(loader):
    model.eval()
    total_loss = 0
    y_true, y_pred = [], []
    with torch.no_grad():
        for data in loader:
            data = data.to(device)
            data.x = data.x.float()
            data.y = data.y.float()

            out = model(data.x, data.edge_index, data.batch)
            loss = criterion(out, data.y.view(-1, 1))
            total_loss += loss.item()

            y_true.append(data.y.cpu())
            y_pred.append(out.cpu())

    y_true = torch.cat(y_true, dim=0)
    y_pred = torch.cat(y_pred, dim=0)

    mse = mean_squared_error(y_true.numpy(), y_pred.numpy())
    rmse = torch.sqrt(torch.tensor(mse))  
    mae = mean_absolute_error(y_true.numpy(), y_pred.numpy())

    return total_loss / len(loader), rmse.item(), mae  

In [12]:
for epoch in range(1, 31):
    train_loss = train()
    val_loss, val_rmse, val_mae = evaluate(val_loader)
    print(f"Epoch: {epoch:02d}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val RMSE: {val_rmse:.4f}, Val MAE: {val_mae:.4f}")

test_loss, test_rmse, test_mae = evaluate(test_loader)
print(f"Test Loss: {test_loss:.4f}, Test RMSE: {test_rmse:.4f}, Test MAE: {test_mae:.4f}")


Epoch: 01, Train Loss: 12.8582, Val Loss: 11.0460, Val RMSE: 3.3555, Val MAE: 2.7546
Epoch: 02, Train Loss: 8.6751, Val Loss: 7.6539, Val RMSE: 2.7967, Val MAE: 2.2386
Epoch: 03, Train Loss: 6.2893, Val Loss: 5.5190, Val RMSE: 2.3756, Val MAE: 1.9046
Epoch: 04, Train Loss: 4.7920, Val Loss: 3.9615, Val RMSE: 2.0134, Val MAE: 1.6187
Epoch: 05, Train Loss: 4.1408, Val Loss: 3.3604, Val RMSE: 1.8535, Val MAE: 1.4846
Epoch: 06, Train Loss: 3.5440, Val Loss: 2.8584, Val RMSE: 1.7147, Val MAE: 1.3518
Epoch: 07, Train Loss: 3.2259, Val Loss: 2.5786, Val RMSE: 1.6344, Val MAE: 1.2994
Epoch: 08, Train Loss: 3.0202, Val Loss: 2.6459, Val RMSE: 1.6516, Val MAE: 1.3360
Epoch: 09, Train Loss: 2.8097, Val Loss: 2.0016, Val RMSE: 1.4503, Val MAE: 1.1116
Epoch: 10, Train Loss: 2.6614, Val Loss: 2.3789, Val RMSE: 1.5735, Val MAE: 1.2340
Epoch: 11, Train Loss: 2.3906, Val Loss: 2.7865, Val RMSE: 1.7058, Val MAE: 1.3733
Epoch: 12, Train Loss: 2.4222, Val Loss: 1.9366, Val RMSE: 1.4144, Val MAE: 1.0968
Ep

In [13]:
import torch.optim as optim
from sklearn.metrics import mean_squared_error, mean_absolute_error
import torch.nn.functional as F

criterion = torch.nn.MSELoss()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = GIN(
    in_channels=dataset.num_node_features,
    hidden_channels=64,
    out_channels=1,  
    num_layers=3,
    dropout=0.5
).to(device)

optimizer = optim.Adam(model.parameters(), lr=0.001)

# 학습 함수
def train():
    model.train()
    total_loss = 0
    for data in train_loader:
        data = data.to(device)
        data.x = data.x.float()
        data.y = data.y.float()

        optimizer.zero_grad()
        out = model(data.x, data.edge_index, data.batch)
        loss = criterion(out, data.y.view(-1, 1))  
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(train_loader)

import torch
from sklearn.metrics import mean_squared_error, mean_absolute_error

def evaluate(loader):
    model.eval()
    total_loss = 0
    y_true, y_pred = [], []
    with torch.no_grad():
        for data in loader:
            data = data.to(device)
            data.x = data.x.float()
            data.y = data.y.float()

            out = model(data.x, data.edge_index, data.batch)
            loss = criterion(out, data.y.view(-1, 1))
            total_loss += loss.item()

            y_true.append(data.y.cpu())
            y_pred.append(out.cpu())

    y_true = torch.cat(y_true, dim=0)
    y_pred = torch.cat(y_pred, dim=0)

    mse = mean_squared_error(y_true.numpy(), y_pred.numpy())
    rmse = torch.sqrt(torch.tensor(mse))  
    mae = mean_absolute_error(y_true.numpy(), y_pred.numpy())

    return total_loss / len(loader), rmse.item(), mae  


for epoch in range(1, 31):
    train_loss = train()
    val_loss, val_rmse, val_mae = evaluate(val_loader)
    print(f"Epoch: {epoch:02d}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val RMSE: {val_rmse:.4f}, Val MAE: {val_mae:.4f}")

test_loss, test_rmse, test_mae = evaluate(test_loader)
print(f"Test Loss: {test_loss:.4f}, Test RMSE: {test_rmse:.4f}, Test MAE: {test_mae:.4f}")



Epoch: 01, Train Loss: 11.9896, Val Loss: 8.2978, Val RMSE: 2.9092, Val MAE: 2.3301
Epoch: 02, Train Loss: 7.3944, Val Loss: 7.7660, Val RMSE: 2.8261, Val MAE: 2.3144
Epoch: 03, Train Loss: 4.7441, Val Loss: 11.6998, Val RMSE: 3.4459, Val MAE: 2.8806
Epoch: 04, Train Loss: 3.0667, Val Loss: 2.4060, Val RMSE: 1.5956, Val MAE: 1.1932
Epoch: 05, Train Loss: 2.1827, Val Loss: 1.8995, Val RMSE: 1.4234, Val MAE: 1.0664
Epoch: 06, Train Loss: 2.0153, Val Loss: 1.9209, Val RMSE: 1.4058, Val MAE: 1.0537
Epoch: 07, Train Loss: 1.8907, Val Loss: 1.3925, Val RMSE: 1.2154, Val MAE: 0.9413
Epoch: 08, Train Loss: 1.9068, Val Loss: 1.4911, Val RMSE: 1.2554, Val MAE: 0.9370
Epoch: 09, Train Loss: 1.7552, Val Loss: 3.1630, Val RMSE: 1.8150, Val MAE: 1.4639
Epoch: 10, Train Loss: 1.7963, Val Loss: 9.8948, Val RMSE: 3.1743, Val MAE: 2.6408
Epoch: 11, Train Loss: 1.6700, Val Loss: 6.2972, Val RMSE: 2.5719, Val MAE: 2.1460
Epoch: 12, Train Loss: 1.7152, Val Loss: 3.4048, Val RMSE: 1.8551, Val MAE: 1.5067
Ep

In [14]:
import torch.optim as optim
from sklearn.metrics import mean_squared_error, mean_absolute_error
import torch.nn.functional as F

criterion = torch.nn.MSELoss()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = GAT(
    in_channels=dataset.num_node_features,
    hidden_channels=64,
    out_channels=1,  
    num_layers=3,
    dropout=0.5
).to(device)

optimizer = optim.Adam(model.parameters(), lr=0.001)

def train():
    model.train()
    total_loss = 0
    for data in train_loader:
        data = data.to(device)
        data.x = data.x.float()
        data.y = data.y.float()

        optimizer.zero_grad()
        out = model(data.x, data.edge_index, data.batch)
        loss = criterion(out, data.y.view(-1, 1))  
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(train_loader)

import torch
from sklearn.metrics import mean_squared_error, mean_absolute_error

def evaluate(loader):
    model.eval()
    total_loss = 0
    y_true, y_pred = [], []
    with torch.no_grad():
        for data in loader:
            data = data.to(device)
            data.x = data.x.float()
            data.y = data.y.float()

            out = model(data.x, data.edge_index, data.batch)
            loss = criterion(out, data.y.view(-1, 1))
            total_loss += loss.item()

            y_true.append(data.y.cpu())
            y_pred.append(out.cpu())

    y_true = torch.cat(y_true, dim=0)
    y_pred = torch.cat(y_pred, dim=0)

    mse = mean_squared_error(y_true.numpy(), y_pred.numpy())
    rmse = torch.sqrt(torch.tensor(mse))  
    mae = mean_absolute_error(y_true.numpy(), y_pred.numpy())

    return total_loss / len(loader), rmse.item(), mae  


for epoch in range(1, 31):
    train_loss = train()
    val_loss, val_rmse, val_mae = evaluate(val_loader)
    print(f"Epoch: {epoch:02d}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val RMSE: {val_rmse:.4f}, Val MAE: {val_mae:.4f}")

test_loss, test_rmse, test_mae = evaluate(test_loader)
print(f"Test Loss: {test_loss:.4f}, Test RMSE: {test_rmse:.4f}, Test MAE: {test_mae:.4f}")



Epoch: 01, Train Loss: 13.0838, Val Loss: 11.1945, Val RMSE: 3.3744, Val MAE: 2.7792
Epoch: 02, Train Loss: 9.3297, Val Loss: 8.1729, Val RMSE: 2.8852, Val MAE: 2.3547
Epoch: 03, Train Loss: 6.9287, Val Loss: 5.7426, Val RMSE: 2.4191, Val MAE: 1.9669
Epoch: 04, Train Loss: 5.2070, Val Loss: 4.5182, Val RMSE: 2.1396, Val MAE: 1.7525
Epoch: 05, Train Loss: 4.6013, Val Loss: 3.8227, Val RMSE: 1.9613, Val MAE: 1.6191
Epoch: 06, Train Loss: 4.2263, Val Loss: 3.3533, Val RMSE: 1.8346, Val MAE: 1.5235
Epoch: 07, Train Loss: 3.9601, Val Loss: 3.1117, Val RMSE: 1.7663, Val MAE: 1.4714
Epoch: 08, Train Loss: 3.7599, Val Loss: 2.8941, Val RMSE: 1.6941, Val MAE: 1.4101
Epoch: 09, Train Loss: 3.3453, Val Loss: 2.6734, Val RMSE: 1.6351, Val MAE: 1.3650
Epoch: 10, Train Loss: 3.2113, Val Loss: 2.4482, Val RMSE: 1.5691, Val MAE: 1.2844
Epoch: 11, Train Loss: 3.0806, Val Loss: 2.6269, Val RMSE: 1.6259, Val MAE: 1.3311
Epoch: 12, Train Loss: 3.0031, Val Loss: 2.1242, Val RMSE: 1.4627, Val MAE: 1.1810
Ep

In [15]:
import torch.optim as optim
from sklearn.metrics import mean_squared_error, mean_absolute_error
import torch.nn.functional as F

criterion = torch.nn.MSELoss()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = GT(
    in_channels=dataset.num_node_features,
    hidden_channels=64,
    out_channels=1,  
    num_layers=3,
    dropout=0.5
).to(device)

optimizer = optim.Adam(model.parameters(), lr=0.001)

# 학습 함수
def train():
    model.train()
    total_loss = 0
    for data in train_loader:
        data = data.to(device)
        data.x = data.x.float()
        data.y = data.y.float()

        optimizer.zero_grad()
        out = model(data.x, data.edge_index, data.batch)
        loss = criterion(out, data.y.view(-1, 1))  
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(train_loader)

import torch
from sklearn.metrics import mean_squared_error, mean_absolute_error

def evaluate(loader):
    model.eval()
    total_loss = 0
    y_true, y_pred = [], []
    with torch.no_grad():
        for data in loader:
            data = data.to(device)
            data.x = data.x.float()
            data.y = data.y.float()

            out = model(data.x, data.edge_index, data.batch)
            loss = criterion(out, data.y.view(-1, 1))
            total_loss += loss.item()

            y_true.append(data.y.cpu())
            y_pred.append(out.cpu())

    y_true = torch.cat(y_true, dim=0)
    y_pred = torch.cat(y_pred, dim=0)

    mse = mean_squared_error(y_true.numpy(), y_pred.numpy())
    rmse = torch.sqrt(torch.tensor(mse))  
    mae = mean_absolute_error(y_true.numpy(), y_pred.numpy())

    return total_loss / len(loader), rmse.item(), mae  


for epoch in range(1, 31):
    train_loss = train()
    val_loss, val_rmse, val_mae = evaluate(val_loader)
    print(f"Epoch: {epoch:02d}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val RMSE: {val_rmse:.4f}, Val MAE: {val_mae:.4f}")

test_loss, test_rmse, test_mae = evaluate(test_loader)
print(f"Test Loss: {test_loss:.4f}, Test RMSE: {test_rmse:.4f}, Test MAE: {test_mae:.4f}")



Epoch: 01, Train Loss: 10.9462, Val Loss: 10.6572, Val RMSE: 3.2939, Val MAE: 2.7016
Epoch: 02, Train Loss: 7.1088, Val Loss: 6.2751, Val RMSE: 2.5252, Val MAE: 2.0326
Epoch: 03, Train Loss: 5.0210, Val Loss: 4.0133, Val RMSE: 2.0204, Val MAE: 1.6379
Epoch: 04, Train Loss: 3.4643, Val Loss: 3.2341, Val RMSE: 1.8198, Val MAE: 1.4383
Epoch: 05, Train Loss: 2.9508, Val Loss: 2.7045, Val RMSE: 1.6633, Val MAE: 1.3030
Epoch: 06, Train Loss: 2.6867, Val Loss: 2.6436, Val RMSE: 1.6423, Val MAE: 1.3032
Epoch: 07, Train Loss: 2.5860, Val Loss: 2.7761, Val RMSE: 1.6863, Val MAE: 1.3469
Epoch: 08, Train Loss: 2.3697, Val Loss: 2.2703, Val RMSE: 1.5194, Val MAE: 1.1966
Epoch: 09, Train Loss: 2.4803, Val Loss: 2.0497, Val RMSE: 1.4497, Val MAE: 1.1459
Epoch: 10, Train Loss: 2.1482, Val Loss: 1.8838, Val RMSE: 1.3932, Val MAE: 1.0740
Epoch: 11, Train Loss: 2.1176, Val Loss: 2.2881, Val RMSE: 1.5396, Val MAE: 1.1928
Epoch: 12, Train Loss: 2.2578, Val Loss: 2.0981, Val RMSE: 1.4709, Val MAE: 1.1400
Ep